In [0]:
import os
import json
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation, Embedding

Using TensorFlow backend.


In [0]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
FOLDERNAME ='Music-Generation-Using-Deep-Learning-master/'

assert FOLDERNAME is not None, "[!] Enter the foldername."

%cd drive/My\ Drive
%cd $FOLDERNAME

data_directory = "Data2/"
data_file = "Data_creazy.txt"
charIndex_json = "char_to_index.json"
model_weights_directory = 'Data2/Model_Weights/'
BATCH_SIZE = 16
SEQ_LENGTH = 64

Mounted at /content/drive
/content/drive/My Drive
/content/drive/My Drive/Music-Generation-Using-Deep-Learning-master


In [0]:
def make_model(unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (1, 1))) 
  
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, stateful = True)) 
    model.add(Dropout(0.2))
    
    model.add((Dense(unique_chars)))
    model.add(Activation("softmax"))
    
    return model

In [0]:
def generate_sequence(epoch_num, initial_index, seq_length):
    with open(os.path.join(data_directory, charIndex_json)) as f:
        char_to_index = json.load(f)
    index_to_char = {i:ch for ch, i in char_to_index.items()}
    unique_chars = len(index_to_char)
    '''
    model = make_model(unique_chars)
    model.load_weights(model_weights_directory + "Weights_Jazz_{}.h5".format(epoch_num))
    '''

    model_Jazz = make_model(unique_chars)
    model_Jazz.load_weights(model_weights_directory + "Weights_Jazz_{}.h5".format(epoch_num))
    W_Jazz = model_Jazz.get_weights() 
    model_creazy = make_model(unique_chars-6)
    model_creazy.load_weights(model_weights_directory + "Weights_creazy_{}.h5".format(epoch_num))
    W_creazy = model_creazy.get_weights() 
    #print(np.shape(W_Classic[0][:44]))
    #print(np.shape(W_Jazz[0]))
    W = W_Jazz
    for i in range(np.shape(W_Jazz)[0]):
      W_creazy_reshape = W_creazy[i]
      W_Jazz_reshape = W_Jazz[i][:np.shape(W_creazy[i])[0]]
      if len(np.shape(W_creazy[i])) == 2 and np.shape(W_Jazz_reshape)[1] != np.shape(W_creazy_reshape)[1]:
        W_Jazz_reshape = []
        for j in range(np.shape(W_creazy[i])[0]):
          W_Jazz_reshape.append(W_Jazz[i][j][:np.shape(W_creazy[i])[1]])
      print(len(np.shape(W_Jazz[i])))
      print(np.shape(W_creazy_reshape))
      print(np.shape(W_Jazz_reshape))
      new_W = [0.5*x+0.5*y for x, y in zip(W_creazy[i],W_Jazz_reshape )]
      W[i] = new_W
    model_creazy.set_weights(W)
     
    sequence_index = [initial_index]
    
    for _ in range(seq_length):
        batch = np.zeros((1, 1))
        batch[0, 0] = sequence_index[-1]
        
        #predicted_probs = model.predict_on_batch(batch).ravel()
        predicted_probs = model_Jazz.predict_on_batch(batch).ravel()
        sample = np.random.choice(range(unique_chars), size = 1, p = predicted_probs)
        
        sequence_index.append(sample[0])
    
    seq = ''.join(index_to_char[c] for c in sequence_index)
    
    cnt = 0
    for i in seq:
        cnt += 1
        if i == "\n":
            break
    seq1 = seq[cnt:]
    
    cnt = 0
    for i in seq1:
        cnt += 1
        if i == "\n" and seq1[cnt] == "\n":
            break
    seq2 = seq1[:cnt]
    
    return seq2

In [0]:
epoch_num = 90
with open(os.path.join(data_directory, charIndex_json)) as f:
        char_to_index = json.load(f)
index_to_char = {i:ch for ch, i in char_to_index.items()}
unique_chars = len(index_to_char)
print(unique_chars)
    
model_Jazz = make_model(unique_chars)
model_Jazz.load_weights(model_weights_directory + "Weights_Jazz_{}.h5".format(epoch_num))
W_Jazz = model_Jazz.get_weights() 
model_creazy = make_model(unique_chars-6)
model_creazy.load_weights(model_weights_directory + "Weights_creazy_{}.h5".format(epoch_num))
W_creazy = model_creazy.get_weights() 
#print(np.shape(W_Classic[0][:44]))
#print(np.shape(W_Jazz[0]))
W = W_Jazz
for i in range(np.shape(W_Jazz)[0]):
  W_creazy_reshape = W_creazy[i]
  W_Jazz_reshape = W_Jazz[i][:np.shape(W_creazy[i])[0]]
  if len(np.shape(W_creazy[i])) == 2 and np.shape(W_Jazz_reshape)[1] != np.shape(W_creazy_reshape)[1]:
    W_Jazz_reshape = []
    for j in range(np.shape(W_creazy[i])[0]):
      W_Jazz_reshape.append(W_Jazz[i][j][:np.shape(W_creazy[i])[1]])
  print(len(np.shape(W_Jazz[i])))
  print(np.shape(W_creazy_reshape))
  print(np.shape(W_Jazz_reshape))
  new_W = [0.5*x+0.5*y for x, y in zip(W_creazy[i],W_Jazz_reshape )]
  W[i] = new_W
model_creazy.set_weights(W)
#model2.set_weights(new_W)
#print(W1)
#print(np.shape(W2[0][:44]))
#print(np.shape(W1))
#print(np.shape(new_W))
#print(new_W)
#print((np.asarray(W1)*0.5+np.asarray(W2)*0.5))

44
2
(38, 512)
(38, 512)
2
(512, 1024)
(512, 1024)
2
(256, 1024)
(256, 1024)
1
(1024,)
(1024,)
2
(256, 1024)
(256, 1024)
2
(256, 1024)
(256, 1024)
1
(1024,)
(1024,)
2
(256, 1024)
(256, 1024)
2
(256, 1024)
(256, 1024)
1
(1024,)
(1024,)
2
(256, 38)
(256, 38)
1
(38,)
(38,)


In [0]:
ep = int(input("1. Which epoch number weight you want to load into the model(10, 20, 30, ..., 90). Small number will generate more errors in music: "))
ar = int(input("\n2. Enter any number between 0 to 46 (pick 32 )which will be given as initial charcter to model for generating sequence: "))
ln = int(input("\n3. Enter the length of music sequence you want to generate. Typical number is between 300-600. Too small number will generate hardly generate any sequence: "))

music = generate_sequence(ep, ar, ln)

print("\nMUSIC SEQUENCE GENERATED: \n",file=open("output.txt", "a"))

print(music, file=open("output.txt", "a"))
print("\nMUSIC SEQUENCE GENERATED: \n")

print(music)
#print("I have a question.", file=open("output.txt", "a"))

1. Which epoch number weight you want to load into the model(10, 20, 30, ..., 90). Small number will generate more errors in music: 90

2. Enter any number between 0 to 46 (pich 32 )which will be given as initial charcter to model for generating sequence: 35

3. Enter the length of music sequence you want to generate. Typical number is between 300-600. Too small number will generate hardly generate any sequence: 500
2
(38, 512)
(38, 512)
2
(512, 1024)
(512, 1024)
2
(256, 1024)
(256, 1024)
1
(1024,)
(1024,)
2
(256, 1024)
(256, 1024)
2
(256, 1024)
(256, 1024)
1
(1024,)
(1024,)
2
(256, 1024)
(256, 1024)
2
(256, 1024)
(256, 1024)
1
(1024,)
(1024,)
2
(256, 38)
(256, 38)
1
(38,)
(38,)

MUSIC SEQUENCE GENERATED: 

[E,2^F,,2=F,2C2]>E,2 [F,,,4E,4^F,,4=B,,,4C4=F,4F,4] C4 [F,,,0E,0^C,,0^F,,0E,0] A,2>_A,,2>E,2 | [F,,,0E,0^F,,0=B,,,0=c0A,0=F0F,0F,0]< [C0A0F0] [d,,0E,0^F,,0D,,0=F,0C0]E, [F,0C,,0E,0^F,,0B,,,0] C4 | [F,,0E,0D,,0^F,,0^C,,
